In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('movies.csv')
df.describe()

,movieId
count,27278.000000
mean,59855.480570
std,44429.314697
min,1.000000
25%,6931.250000
50%,68068.000000
75%,100293.250000
max,131262.000000


In [3]:
df['title'] = [title.lower()for title in df['title']]
df['title'] = [re.sub('\([0-9]{4}\)', '',  title) for title in df['title']]
df.head()

,movieId,title,genres
0,1,toy story,Adventure|Animation|Children|Comedy|Fantasy
1,2,jumanji,Adventure|Children|Fantasy
2,3,grumpier old men,Comedy|Romance
3,4,waiting to exhale,Comedy|Drama|Romance
4,5,father of the bride part ii,Comedy


In [4]:
def getDtm(textList):
    vect = CountVectorizer(ngram_range=(1,1))
    vect.fit(textList)
    return vect.transform(textList)

In [5]:
def getM(textList, query):
    vect = CountVectorizer(ngram_range=(1,1))
    vect.fit(textList)
    return vect.transform(query)

In [6]:
def getQ(textList, m, query, c=2):
    sumM = getSumM(textList, query)
    
    alpha = c*m
    alphaL = alpha + sumM
    beta = c*(1-m)
    betaL = beta + len(textList) - sumM
    
    a = np.log(alphaL)
    a2 = np.log(alpha)
    b = np.log(betaL)
    b2 = np.log(beta)
    
    return a-a2-b+b2

In [7]:
query = ['toy sotry', 'old men']
dtm = getDtm(df['title'])
dtm.shape

(27278, 21887)

In [8]:
x = getM(df['title'], query)
x.todense()
x.shape

(2, 21887)

In [9]:
alpha = 2*np.mean(dtm, axis=0)
beta = 2*(1-np.mean(dtm, axis=0))
beta.shape

(1, 21887)

In [10]:
alphaL = alpha + x.sum(axis=0)
betaL = beta + len(query) - x.sum(axis=0)
betaL.shape

(1, 21887)

In [11]:
q = np.log(alphaL) - np.log(alpha) - np.log(betaL) + np.log(beta)
q.shape

(1, 21887)

In [12]:
nc = np.log(alpha + beta) - np.log(alpha + beta + len(query)) + np.log(betaL) - np.log(beta)
nc
nc.sum(axis=0)


matrix([[3.30099583e-04, 1.83302936e-05, 1.83302936e-05, ...,
         1.83302936e-05, 1.83302936e-05, 1.83302936e-05]])

In [18]:
nc

matrix([[3.30099583e-04, 1.83302936e-05, 1.83302936e-05, ...,
         1.83302936e-05, 1.83302936e-05, 1.83302936e-05]])

In [24]:
dtm.shape

(27278, 21887)

In [31]:
dtm*q.transpose()

matrix([[ 5.77931536],
        [-0.69316551],
        [ 8.74296626],
        ...,
        [-1.62229935],
        [-1.38633102],
        [-0.69338555]])